# Fake Face Detection

In [1]:
import cv2
import os
import time
import sys
from matplotlib import pyplot as plt 
import numpy as np
from collections import deque

print("OpenCV version",cv2.__version__)
print("Numpy version",np.__version__)


OpenCV version 4.4.0
Numpy version 1.17.2


## Import Dataset

In [2]:
def import_images(percent_split):
    # initiate img types and path
    img_types = ["C", "L", "R"]
    dataset_size = 0
    
    _, _, fake_data_count = next(os.walk("./fake"))
    dataset_size += len(fake_data_count) - 1
    fake_data_count = (len(fake_data_count) - 1)// 3
    
    _, _, real_data_count = next(os.walk("./real"))
    dataset_size += len(real_data_count) - 1
    real_data_count = (len(real_data_count) - 1) // 3

    real_imgs = [
        [cv2.imread(f"./real/{i}{t}r.jpg") for t in img_types]
        for i in range(1, real_data_count + 1)
    ]
    fake_imgs = [
        [cv2.imread(f"./fake/{i}{t}f.jpg") for t in img_types]
        for i in range(1, fake_data_count + 1)
    ]

    real_split_idx = int(real_data_count*percent_split)
    fake_split_idx = int(fake_data_count*percent_split)
    
    real_test_count = len(real_imgs[real_split_idx:])
    fake_test_count = len(fake_imgs[fake_split_idx:])
    
    test_imgs = real_imgs[real_split_idx:]+fake_imgs[fake_split_idx:]
    real_imgs = real_imgs[:real_split_idx]
    fake_imgs = fake_imgs[:fake_split_idx]
    
    return real_imgs, fake_imgs, test_imgs, real_test_count, fake_test_count, dataset_size


## Haar Cascade Face Finder

In [3]:
class FaceFinder:
    def __init__(self, path) -> None:

        # load frontal face detector files
        cv2.samples.addSamplesDataSearchPath(path)
        front_face_cascade_name = "haarcascade_frontalface_default.xml"

        # load frontal alt detector files
        cv2.samples.addSamplesDataSearchPath(path)
        alt_face_cascade_name = "haarcascade_frontalface_alt.xml"

        # load frontal alt2 detector files
        cv2.samples.addSamplesDataSearchPath(path)
        alt2_face_cascade_name = "haarcascade_frontalface_alt2.xml"

        # load frontal profile detector files
        cv2.samples.addSamplesDataSearchPath(path)
        profile_face_cascade_name = "haarcascade_profileface.xml"

        # create and load the frontal face cascade
        self.frontal_face_cascade = cv2.CascadeClassifier()
        if not self.frontal_face_cascade.load(
            cv2.samples.findFile(front_face_cascade_name)
        ):
            print("--(!)Error loading frontal face cascade")
            exit(0)

        # create and load the alt face cascade
        self.alt_face_cascade = cv2.CascadeClassifier()
        if not self.alt_face_cascade.load(cv2.samples.findFile(alt_face_cascade_name)):
            print("--(!)Error loading alt face cascade")
            exit(0)

        # create and load the alt2 face cascade
        self.alt2_face_cascade = cv2.CascadeClassifier()
        if not self.alt2_face_cascade.load(
            cv2.samples.findFile(alt2_face_cascade_name)
        ):
            print("--(!)Error loading alt2 face cascade")
            exit(0)

        # create and load the profile face cascade
        self.profile_face_cascade = cv2.CascadeClassifier()
        if not self.profile_face_cascade.load(
            cv2.samples.findFile(profile_face_cascade_name)
        ):
            print("--(!)Error loading profile face cascade")
            exit(0)

    def ultimate_find_face(self, frame):
        face = self.find_frontal_face(frame)
        if face is not None:
            return face
        face = self.find_alt_face(frame)
        if face is not None:
            return face
        face = self.find_alt2_face(frame)
        if face is not None:
            return face
        face = self.find_profile_face(frame)
        return face

    def find_frontal_face(self, frame):
        # get the first face in the image
        faces = self.frontal_face_cascade.detectMultiScale(frame)
        if len(faces) == 0:
            return None
        face = self.frontal_face_cascade.detectMultiScale(frame)[0]
        x, y, width, height = face

        # crop the picture so it only includes that face
        face = frame[y : y + height, x : x + width]
        return face

    def find_alt_face(self, frame):
        # get the first face in the image
        faces = self.alt_face_cascade.detectMultiScale(frame)
        if len(faces) == 0:
            return None
        face = self.alt_face_cascade.detectMultiScale(frame)[0]
        x, y, width, height = face

        # crop the picture so it only includes that face
        face = frame[y : y + height, x : x + width]
        return face

    def find_alt2_face(self, frame):
        # get the first face in the image
        faces = self.alt2_face_cascade.detectMultiScale(frame)
        if len(faces) == 0:
            return None
        face = self.alt2_face_cascade.detectMultiScale(frame)[0]
        x, y, width, height = face

        # crop the picture so it only includes that face
        face = frame[y : y + height, x : x + width]
        return face

    def find_profile_face(self, frame):
        # get the first face in the image
        faces = self.profile_face_cascade.detectMultiScale(frame)
        if len(faces) == 0:
            return None
        face = self.profile_face_cascade.detectMultiScale(frame)[0]
        x, y, width, height = face

        # crop the picture so it only includes that face
        face = frame[y : y + height, x : x + width]
        return face

    def detect_triplet(self, triplet):
        center, left, right = triplet
        center_face = self.ultimate_find_face(center)
        left_face = self.ultimate_find_face(left)
        right_face = self.ultimate_find_face(right)
        return [
            center_face,
            left_face,
            right_face,
        ]


## Training

In [4]:
def matches(img1, img2):
    my_SIFT_instance = cv2.SIFT_create()
    kp1, des1 = my_SIFT_instance.detectAndCompute(img1, None)
    kp2, des2 = my_SIFT_instance.detectAndCompute(img2, None)
    lowes = cv2.FlannBasedMatcher().knnMatch(
        np.asarray(des1, np.float32), np.asarray(des2, np.float32), k=2
    )

    leftPoints, rightPoints = [], []
    for m, n in lowes:
        if (m.distance / n.distance) < 0.8:
            rightPoints.append(kp2[m.trainIdx].pt)
            leftPoints.append(kp1[m.queryIdx].pt)

    leftPoints, rightPoints = np.int32(leftPoints), np.int32(rightPoints)

    return len(leftPoints)

def get_triplet_match(triplet, show_imgs=False):
    if show_imgs:
        show_img(triplet, names=["C", "L", "R"])

    center, left, right = triplet
    if center is None:
        return matches(left, right)
    if left is None:
        return matches(center, right)
    if right is None:
        return matches(center, left)

    sizes = [(img.shape[0] * img.shape[1], img) for img in triplet]
    try:
        sizes.remove(min(sizes))
    except:
        sizes = [sizes[0], sizes[1]]

    return matches(sizes[0][1], sizes[1][1])

def train(real_imgs, fake_imgs, face_finder):
    real_faces = [face_finder.detect_triplet(triplet) for triplet in real_imgs]
    fake_faces = [face_finder.detect_triplet(triplet) for triplet in fake_imgs]
    
    real_face_matches = [
        get_triplet_match(real_face_triplet) for real_face_triplet in real_faces
    ]
    
    fake_face_matches = [
        get_triplet_match(fake_face_triplet) for fake_face_triplet in fake_faces
    ]
    match_threshold = (max(real_face_matches) + min(fake_face_matches)) // 2
    
    return match_threshold

def detect(triplet, threshold):
    matches = get_triplet_match(triplet, show_imgs=False)
    if matches < threshold:
        return True
    return False


In [5]:
real_imgs, fake_imgs, test_imgs, real_label_count, fake_label_count, dataset_size = import_images(0.7)
path = "./res"
face_finder = FaceFinder(path)
true_labels = [True for _ in range(real_label_count)]+[False for _ in range(fake_label_count)]

train_start = time.time()
match_threshold = train(real_imgs, fake_imgs, face_finder)
train_end = time.time()
test_start = time.time()
test_faces = [face_finder.detect_triplet(triplet) for triplet in test_imgs]
results = [detect(triplet, match_threshold) for triplet in test_faces]
test_end = time.time()


# Results

In [6]:
true_pos, true_neg, false_pos, false_neg = 0, 0, 0, 0

for i in range(len(results)):
    if true_labels[i] == True and results[i] == True: true_pos += 1
    elif true_labels[i] == False and results[i] == False: true_neg += 1
    elif true_labels[i] == False and results[i] == True: false_pos += 1
    elif true_labels[i] == True and results[i] == False: false_neg += 1

precision = true_pos / (true_pos + false_pos) * 100
recall = true_pos / (true_pos + false_neg) * 100

print('Precision: %.2f%%' % precision)
print('Recall: %.2f%%'% recall)
print('Dataset Size: %.2f ' % dataset_size)
print('Training: %.2f s ' % (train_end - train_start))
print('Testing: %.2f s ' % (test_end - test_start))

# print confusion matrix
confusion_matrix = f"\n\n\tPositive\tNegative\n----------------------------------\nTrue\t\t{true_pos}\t\t{true_neg}\nFalse\t\t{false_pos}\t\t{false_neg}"
print(confusion_matrix)


Precision: 100.00%
Recall: 100.00%
Dataset Size 72.00: 
Training 21.47 s: 
Testing 12.98 s: 


	Positive	Negative
----------------------------------
True		4		4
False		0		0
